# Import and install Dependencies

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Create Keypoints using Mediapipe

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        #  # Flip the frame horizontally
        # frame = cv2.flip(frame, 1)

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

         # Resize and show the image
        image = cv2.resize(image, (1000,800))

        # Show to screen
        cv2.imshow('SignPal', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(cv2.flip(image,1), cv2.COLOR_BGR2RGB)) 

# Extract keypoint values

In [ ]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [ ]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [ ]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)


In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
result_test = extract_keypoints(results)

In [ ]:
result_test

In [ ]:
np.save('0', result_test)

In [ ]:
np.load('0.npy')

# Setup Folders for Data Collection

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
# actions = np.array(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','A','Afternoon','Ayubowan','B','C','D','E','Evening','F','Friday','G','Good','H','I','J','Monday', 'Morning', 'Night','Noon','Saturday','Sunday','Thursday','Tuesday','Wednesday'])
actions = np.array(['1','2','3','Afternoon','Ayubowan','Evening','Good']) 

# Save the labels to a text file
with open('dataset_label.txt', 'w') as f:
    for label in actions:
        f.write(f"{label}\n")

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# # Ensure the directory structure exists
# for action in actions:
#     action_path = os.path.join(DATA_PATH, action)
#     if not os.path.exists(action_path):
#         os.makedirs(action_path)  # Create folder if it does not exist
#         print(f"Created folder: {action_path}")

# Folder start
# start_folder = 0
# for action in actions: 
#     dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int)) if os.listdir(os.path.join(DATA_PATH, action)) else 0
#     for sequence in range(1, no_sequences + 1):
#         try: 
#             os.makedirs(os.path.join(DATA_PATH, action, str(dirmax + sequence)))
#         except:
#             pass

In [ ]:
for action in actions: 
    # dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass
# for action in actions:
#     action_path = os.path.join(DATA_PATH, action)
#     if not os.path.exists(action_path):
#         os.makedirs(action_path)
#     dirmax = np.max(np.array(os.listdir(action_path)).astype(int)) if os.listdir(action_path) else 0
#     for sequence in range(1, no_sequences + 1):
#         try:
#             os.makedirs(os.path.join(action_path, str(dirmax + sequence)))
#         except:
#             pass

# Collect keypoint values for training and testing

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,20), 
                               cv2.FONT_HERSHEY_SIMPLEX,0.8, (0, 225, 0), 2, cv2.LINE_AA)

                    # Resize and show the image
                    image = cv2.resize(image, (1000,800))
                    # Show to screen
                    cv2.imshow('SignPal', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,20), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 225, 0), 2, cv2.LINE_AA)

                     # Resize and show the image
                    image = cv2.resize(image, (1000,800))
                    # Show to screen
                    cv2.imshow('SignPal', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                # # Ensure directory exists
                # os.makedirs(os.path.dirname(npy_path), exist_ok=True)
                
                # Save keypoints
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

# Processing and data labeling

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}


In [ ]:
label_map

In [ ]:
# sequences, labels = [], []
# for action in actions:
#     for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
#         window = []
#         for frame_num in range(sequence_length):
#             res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
#             window.append(res)
#         sequences.append(window)
#         labels.append(label_map[action])
sequences, labels = [], [] 
for action in actions: 
    for sequence in range(no_sequences):
        window = []  # This line should be properly aligned with the inner for-loop
        for frame_num in range(sequence_length): 
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
# from sklearn.model_selection import train_test_split

# # Assuming X_data and y_data are your features and labels
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# np.save("X_test.npy", X_test)
# np.save("y_test.npy", y_test)


In [ ]:
y_test.shape

# Build and train LSTM model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# model = Sequential()

# # Increase LSTM units and add more layers
# model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(30, 1662)))
# model.add(Dropout(0.2))  # Prevent overfitting
# model.add(BatchNormalization())  # Normalize activations

# model.add(LSTM(256, return_sequences=True, activation='relu'))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())

# model.add(LSTM(128, return_sequences=False, activation='relu'))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())

# # Increase Dense layer size
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='relu'))

# # Output layer with softmax
# model.add(Dense(actions.shape[0], activation='softmax'))
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()

# Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[4])]

# Save Model

In [ ]:
model.save('my_model.keras')

In [ ]:
# Save the model in .h5 format
model.save('my_model.h5')


In [ ]:
del model

In [ ]:
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
model = load_model('my_model.h5')


In [ ]:
model.load_weights('my_model.keras')

In [ ]:
import tensorflow as tf
import pickle

# Load the Keras model from the .h5 file
model = tf.keras.models.load_model('my_model.keras')

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Enable resource variables
converter.experimental_enable_resource_variables = True

# Use select TensorFlow ops
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

# Disable lowering of tensor list ops
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()

# Save the converted .tflite model to a file
with open('action_model.tflite', 'wb') as f:
    f.write(tflite_model)

# Save the .tflite model in .pkl format
with open('action_model.pkl', 'wb') as pkl_file:
    pickle.dump(tflite_model, pkl_file)

print("Model saved in .tflite and .pkl formats!")


# conusion matrix,accuracy, classification reports

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import numpy as np

# Evaluate Model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert softmax outputs to class labels
y_true_classes = np.argmax(y_test, axis=1)  # Convert one-hot labels to class labels

# Compute Accuracy, Precision, Recall, F1 Score
print("Classification Report:\n", classification_report(y_true_classes, y_pred_classes))

# Compute Confusion Matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Plot Confusion Matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=actions, yticklabels=actions)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plot Accuracy & Loss Graphs
history = model.history.history

plt.figure(figsize=(12, 5))

# Accuracy Graph
plt.subplot(1, 2, 1)
if 'categorical_accuracy' in history:
    plt.plot(history['categorical_accuracy'], label='Train Accuracy')
else:
    print("Train Accuracy not found in history.")

if 'val_categorical_accuracy' in history:
    plt.plot(history['val_categorical_accuracy'], label='Validation Accuracy')
else:
    print("Validation Accuracy not found in history.")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy Graph')
plt.legend()

# Loss Graph
plt.subplot(1, 2, 2)
if 'loss' in history:
    plt.plot(history['loss'], label='Train Loss')
else:
    print("Train Loss not found in history.")

if 'val_loss' in history:
    plt.plot(history['val_loss'], label='Validation Loss')
else:
    print("Validation Loss not found in history.")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Graph')
plt.legend()

plt.show()

# ROC Curve for Multi-class
plt.figure(figsize=(10, 7))
for i in range(y_test.shape[1]):  # Iterate through each class
    fpr, tpr, _ = roc_curve(y_test[:, i], y_pred[:, i])
    auc_score = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {actions[i]} (AUC = {auc_score:.2f})')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# Testing

In [ ]:
from scipy import stats

In [ ]:
# Initialize an empty sentence to store detected actions
sentence = []
previous_action = None

# colors = [(245,117,16), (117,245,16), (16,117,245)]
# if len(colors) < len(actions):
#     colors = [tuple(np.random.randint(0, 255, 3).tolist()) for _ in range(len(actions))]
# def prob_viz(res, actions, input_frame, colors):
#     output_frame = input_frame.copy()
#     for num, prob in enumerate(res):
#         cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
#         cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
#     return output_frame


# Function to visualize the most likely action and form sentences
def prob_viz(res, actions, input_frame):
    global sentence, previous_action
    output_frame = input_frame.copy()

    # Get the index of the highest probability
    max_prob_index = np.argmax(res)

    # Define the action with the highest probability
    action = actions[max_prob_index]

    # Append the action to the sentence only if it's not repeated
    if action != previous_action:
        sentence.append(action)
        previous_action = action

    # Limit the length of the sentence to avoid overflowing
    if len(sentence) > 5:  # Change 10 to any number you prefer for sentence length
        sentence = sentence[-5:]

    # # Display the sentence at the bottom of the frame
    # cv2.putText(output_frame, ' '.join(sentence), 
    #             (10, output_frame.shape[0] - 20),  # Position it horizontally on the left
    #             cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    ## Join the sentence to form the text to display
    text = ' '.join(sentence)

# Get the text size
    (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)

# Calculate the x-coordinate for center alignment
    x = (output_frame.shape[1] - text_width) // 2

# y-coordinate remains near the bottom
    y = output_frame.shape[0] - 20

# Display the text at the bottom center of the frame
    cv2.putText(output_frame, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image))
plt.show()

In [ ]:
import cv2
import numpy as np

# Initialize variables for sentence formation and predictions
sequence = []
sentence = []
predictions = []
threshold = 0.5

# Function to visualize the most likely action and form sentences
def prob_viz(res, actions, input_frame):
    global sentence
    output_frame = input_frame.copy()
    return output_frame

# Start capturing video
cap = cv2.VideoCapture(0)

# Load the mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read frame from video capture
        ret, frame = cap.read()

        # Make detections using mediapipe
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks on the detected frame
        draw_styled_landmarks(image, results)
        
        # Extract keypoints from the results
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]  # Keep the last 30 frames
        
        # Prediction logic (Make prediction once sequence has 30 frames)
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            
            # Check if the prediction is consistent over the last 10 frames
            if np.unique(predictions[-10:])[0] == np.argmax(res): 
                if res[np.argmax(res)] > threshold:  # Apply the threshold

                    # Only append the action if it's different from the last one
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            # Keep the sentence length limited to the last 5 words
            if len(sentence) > 1: 
                sentence = sentence[-1:]

        # Visualization
        if len(sequence) == 30:
            image = prob_viz(res, actions, image)
        
        # Draw a rectangle at the bottom of the image to display the sentence
        height, width, _ = image.shape
        text = ' '.join(sentence)
        (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        x = (width - text_width) // 2
        y = height - 10
        # Display the text at the bottom center of the frame
        cv2.putText(image, text, (x, y), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Resize and show the image
        image = cv2.resize(image, (1000,800))
        
        # Display the output image
        cv2.imshow('SignPal', image)

        # Break the loop when 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release the capture and close any open windows
cap.release()
cv2.destroyAllWindows()
